In [ ]:
import pandas as pd 

df = pd.read_csv('data.csv')
display(df.head())
print('Total Rows: ', df.shape[0])

In [ ]:
df_train = df.sample(frac=0.9, random_state=8429)
df_unseen = df.drop(df_train.index)

df_train.reset_index(drop=True, inplace=True)
df_unseen.reset_index(drop=True, inplace=True)

print('Samples for modeling:', df_train.shape[0], "rows")
print('Samples of unseen data to test the model:', df_unseen.shape[0],  "rows")

In [ ]:
from pycaret.classification import ClassificationExperiment, load_model, predict_model

exp = ClassificationExperiment()

In [ ]:
from practicuscore import configure_experiment

experiment_name = 'Telco Customer Churn'
configure_experiment(experiment_name=experiment_name, service_name='Mlflow (Primary)')

In [ ]:
setup_params = {}

In [ ]:
exp.setup(data=df_train, target='churn', session_id=8429, 
          log_experiment=True, experiment_name=experiment_name, **setup_params)

In [ ]:
best_model = exp.compare_models()

In [ ]:
# best_model = exp.create_model()

In [ ]:
tune_params = {}

In [ ]:
tuned_model = exp.tune_model(best_model, **tune_params)

In [ ]:
final_model = exp.finalize_model(tuned_model)

In [ ]:
unseen_predictions = exp.predict_model(final_model, data=df_unseen)
display(unseen_predictions)

In [ ]:
exp.save_model(final_model, 'model')

In [ ]:
loaded_model = load_model('model')

unseen_predictions = predict_model(loaded_model, data=df_unseen)
display(unseen_predictions)

In [ ]:
from practicuscore import DataPipeline
dp = DataPipeline()

host_url = 'https://practicus.company.com'
email = 'user@company.com'
prefix = 'models/my-team/test'
deployment_key = 'some_deployment'
model_name = 'my-model'
model_dir = None  # Current dir

In [ ]:
import getpass
print('Password:')
password = getpass.getpass()

In [ ]:
auth_token = dp.get_auth_token(host_url=host_url, email=email, password=password)
print(f'{email} is authenticated' if auth_token else 'Authentication error. Please retry')

In [ ]:
# Please verify that model.json is up to date, especially the model input fields

dp.deploy_model(
    host_url=host_url, email=email, auth_token=auth_token, deployment_key=deployment_key,
    prefix=prefix, model_name=model_name, model_dir=model_dir
)